In [1]:
import numpy as np
import pandas as pd
import random as rd
import time

In [2]:
def InitialPopulation (PopulationSize, InitialSolution):
    '''This function generates the initial population from given initial solution (InitialSolution) 
    where the size of the population is the input (PopulationSize)'''
    X_All=[]
    for i in range(10):
        X_New=rd.sample(InitialSolution, len(InitialSolution))
        X_All.append(X_New)
    return X_All

In [3]:
def Objective(NumberOfMachines, NumberOfJobs, ProcessingTimeDF, Solution):
    '''According to the job-machine processing time data (ProcessingTimeDF) given in pandas dataframe format
    this function calculate the total makespan of given solution (Solution) '''
    Time_Pass=[0]*NumberOfJobs
    for machine in range (NumberOfMachines):
        for job in range(0,NumberOfJobs):
            time_spent=Time_Pass[job]
            if job>0:
                time_spent=max(Time_Pass[job-1], Time_Pass[job])
            Time_Pass[job]= time_spent + ProcessingTimeDF.loc[Solution[job]].to_list()[machine]
    return Time_Pass[-1]#total time pass until the last job is finished

In [4]:
def RouletteWheelParentSelection(All_Objectives):
    '''This function performs parent selection by roulette wheel solution method where the selection is biased
    by the solution that has better objective value'''
    denominator=0
    for obj in All_Objectives:
        denominator+=max(All_Objectives)-obj
        
    Probability={}
    j=0
    for obj in All_Objectives:
        Probability[j]=((max(All_Objectives)-obj)/denominator)
        j+=1
        
    Sorted_Probability={k: v for k, v in sorted(Probability.items(), key=lambda item: item[1])}
    pick = rd.uniform(0, 1)
    current = 0
    for key, value in Sorted_Probability.items():
        current += value
        if current > pick:
            return key

In [5]:
def TwoPointCrossOver(P1, P2):
    '''This function performs cross over operator based on two point cross over.
    Within this two point cross over operator defined, the jobs between two randomly
    selected points  always inherited from one parent to the child,
    and other jobs are placed in order'''
    
    CO_points=rd.sample([i for i in range(1,len(P1))],2)
    CO_points.sort()
    
    InheritFromP1=P1[CO_points[0]:CO_points[1]]
    NewP2=[i for i in P2 if i not in InheritFromP1]
    Child1=[]
    for i1 in range(0,CO_points[0]):
        Child1.extend([NewP2[i1]]) 
    ILast=len(Child1)
    Child1.extend(InheritFromP1)
    for i2 in range (ILast,len(NewP2)):
        Child1.extend([NewP2[i2]])
    
    InheritFromP2=P2[CO_points[0]:CO_points[1]]
    NewP1=[i for i in P1 if i not in InheritFromP2]
    Child2=[]
    for i1 in range(0,CO_points[0]):
        Child2.extend([NewP1[i1]]) 
    ILast=len(Child2)
    Child2.extend(InheritFromP2)
    for i2 in range (ILast,len(NewP1)):
        Child2.extend([NewP1[i2]])
    return Child1, Child2

In [6]:
def Mutation (Child1, Child2, p):
    '''This function perfoms two job exchange mutation to the children (Child1, Child2) 
   with the probability p'''
   
    u=rd.uniform(0,1)
    if u<=p:
        M_points=rd.sample([i for i in range(0,len(Child1))],2)
        i1=Child1[M_points[0]]
        i2=Child1[M_points[1]]
        Child1[M_points[0]]=i2
        Child1[M_points[1]]=i1
    
    u=rd.uniform(0,1)
    if u<=p:
        M_points=rd.sample([i for i in range(0,len(Child2))],2)
        i1=Child2[M_points[0]]
        i2=Child2[M_points[1]]
        Child2[M_points[0]]=i2
        Child2[M_points[1]]=i1
    
    return Child1, Child2

In [7]:
def RouletteWheelReplacementSelection(All_Objectives):
    '''Roulette wheel selection procedure to decide which individual will be drawn from population '''
    denominator=0
    for obj in All_Objectives:
        denominator+=obj-min(All_Objectives)
        
    Probability={}
    j=0
    for obj in All_Objectives:
        Probability[j]=((obj-min(All_Objectives))/denominator)
        j+=1
        
    Sorted_Probability={k: v for k, v in sorted(Probability.items(), key=lambda item: item[1])}
    pick = rd.uniform(0, 1)
    current = 0
    for key, value in Sorted_Probability.items():
        current += value
        if current > pick:
            return key

In [44]:
def GA(InstanceDataFrame, PopulationSize, NumberOfIterations, MutationProbability, BestKnown):
    start_time = time.time()
    
    #Initialization
    j=len(InstanceDataFrame.index) #number of jobs
    m=len(InstanceDataFrame.columns) #number of machines
    X_Initial=[i for i in range(1,j+1)] # initial individual to generate the initial population
    Solutions=InitialPopulation(PopulationSize, X_Initial)
    All_Objectives=[]
    for Solution in Solutions:
        All_Objectives.append(Objective(m,j,InstanceDataFrame, Solution))
        
    Incumbent=min(All_Objectives)
    BestSolution= Solutions[All_Objectives.index(Incumbent)]
    
    BestSolutionWithinChoromosome=[] #to track the average best objective values within choromosomes

    for iteration in range(NumberOfIterations):
        
        #First Parent Selection
        IndexOfP1=RouletteWheelParentSelection(All_Objectives)
        
        #Second Parent Selection, where roulette wheel is performed again by drawing first parent from population
        New_All_Objectives=All_Objectives.copy()
        New_Solutions=Solutions.copy()
        del (New_All_Objectives[IndexOfP1])
        del (New_Solutions[IndexOfP1])
        IndexOfP2=RouletteWheelParentSelection(New_All_Objectives)
    
        P1=Solutions[IndexOfP1]
        P2=New_Solutions[IndexOfP2]
    
        #Crossover
        Child1, Child2=TwoPointCrossOver(P1,P2)
    
        #Mutation
        MChild1, MChild2=Mutation(Child1, Child2, MutationProbability)
    
        #Subtitution
        IndexOfReplacedIndividual1=RouletteWheelReplacementSelection(All_Objectives)
        New_All_Objectives=All_Objectives.copy()
        New_Solutions=Solutions.copy()
        del (New_All_Objectives[IndexOfReplacedIndividual1])
        del (New_Solutions[IndexOfReplacedIndividual1])
        IndexOfReplacedIndividual2=RouletteWheelParentSelection(New_All_Objectives)
        del (New_All_Objectives[IndexOfReplacedIndividual2])
        del (New_Solutions[IndexOfReplacedIndividual2])
    
        #Replacement
    
        New_All_Objectives.append(Objective(m,j,InstanceDataFrame, MChild1))
        New_All_Objectives.append(Objective(m,j,InstanceDataFrame, MChild2))
        New_Solutions.append(MChild1)
        New_Solutions.append(MChild2)
        
        BestSolutionWithinChoromosome.append(min(New_All_Objectives))
    
        #Update incumbent if it is necessary
        if min(New_All_Objectives)<Incumbent:
            Incumbent=min(New_All_Objectives)
            BestSolution=New_Solutions[New_All_Objectives.index(Incumbent)]
    
        #Continue
        All_Objectives=New_All_Objectives
        Solutions=New_Solutions
        
    CPU_time=np.round((time.time() - start_time),2)   
    Avg_Best=np.round(sum(BestSolutionWithinChoromosome)/len(BestSolutionWithinChoromosome),2)
    Gap=np.round(((Incumbent-BestKnown)/BestKnown),2)
    return BestSolution, Incumbent, Avg_Best , CPU_time, Gap

In [11]:
#keep the input as dataframe
ins_j8m4= pd.read_csv('data_car1', header=None, delimiter=";")
ins_j8m4.index +=1
ins_j13m5=pd.read_csv('data_reC05', header=None, delimiter=";")
ins_j13m5.index +=1
ins_j20m5=pd.read_csv('data_reC09', header=None, delimiter=";")
ins_j20m5.index+=1

In [30]:
GA(ins_j8m4,5, 1000, 1, 4534)

([8, 7, 5, 3, 6, 2, 1, 4], 4534, 4911.98, 5.35, 0.0)

In [45]:
GA(ins_j13m5,5, 1000, 1, 920)

([3, 5, 12, 8, 2, 9, 13, 6, 11, 10, 7, 1, 4], 937, 982.71, 13.25, 0.02)

In [48]:
GA(ins_j20m5,5, 1000, 1, 1302)

([19, 16, 4, 14, 13, 12, 10, 9, 17, 5, 18, 3, 7, 1, 20, 8, 6, 15, 11, 2],
 1318,
 1391.63,
 18.73,
 0.01)